# Análisis exploratorio de incidencia delictiva en México

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
df = pd.read_csv('data/delitos.csv')

#### Ejercicio 1: Elige 3 estados de la república y grafica una serie de tiempo de la frecuencia abosluta de homicidios dolosos de enero 2015 a julio 2019 en estas tres entidades

In [ ]:
estados = ['Jalisco', 'CDMX', 'Chihuahua']
df_filtrado = df[df['Entidad'].isin(estados)]
df_filtrado['Fecha'] = pd.to_datetime(df_filtrado['Año'].astype(str) + '-' + df_filtrado['Mes'].astype(str))
plt.figure(figsize=(12,6))
for estado in estados:
    df_estado = df_filtrado[df_filtrado['Entidad'] == estado]
    plt.plot(df_estado['Fecha'], df_estado['Homicidios_dolosos'], label=estado)

plt.title('Homicidios dolosos por mes (Ene 2015 - Jul 2019)')
plt.xlabel('Fecha')
plt.ylabel('Número de homicidios dolosos')
plt.legend()
plt.grid(True)
plt.show()

#### Ejercicio 2: Contetas las siguientes  preguntas:
1. ¿Cuántos homicidios dolosos hubo en Colima en el 2018?
2. ¿Cuantos robos de vehículo automotor ha habido en el 2019?
3. Obten la suma de homicidos dolosos y feminidios en toda la República Mexicana en cada año.
4. ¿En qué mes y en qué municipio ha ocurrido el mayor número de feminicidios?
5. ¿En qué año y en qué estado ha ocurrido el mayor número de feminicidios?

In [ ]:
homicidios_colima_2018 = df[(df['Entidad']=='Colima') & (df['Año']==2018)]['Homicidios_dolosos'].sum()
print("Homicidios dolosos en Colima 2018:", homicidios_colima_2018)

robos_2019 = df[df['Año']==2019]['Robos_vehículo'].sum()
print("Robos de vehículo en 2019:", robos_2019)

df['Homicidios_y_Feminicidios'] = df['Homicidios_dolosos'] + df['Feminicidios']
suma_anual = df.groupby('Año')['Homicidios_y_Feminicidios'].sum()
print("\nSuma anual de homicidios dolosos y feminicidios:\n", suma_anual)

mayor_feminicidios = df.loc[df['Feminicidios'].idxmax()]
print("\nMes y municipio con más feminicidios:")
print(mayor_feminicidios[['Mes','Municipio','Feminicidios']])

df_estado_anual = df.groupby(['Año','Entidad'])['Feminicidios'].sum().reset_index()
max_fem = df_estado_anual.loc[df_estado_anual['Feminicidios'].idxmax()]
print("\nAño y estado con más feminicidios:")
print(max_fem)



#### Ejercicio 3: Haz una gráfica de pastel de tipos de delito. Deberás crear una gráfica para cada año. Utilzia la función subplots de matplotlib

In [ ]:
años = sorted(df['Año'].unique())

fig, axes = plt.subplots(len(años), 1, figsize=(8, 4*len(años)))

for i, año in enumerate(años):
    df_año = df[df['Año'] == año]
    # Sumar por tipo de delito
    delitos_tipo = df_año.groupby('Tipo_delito')['Frecuencia'].sum()
    axes[i].pie(delitos_tipo, labels=delitos_tipo.index, autopct='%1.1f%%')
    axes[i].set_title(f'Tipos de delito en {año}')

plt.tight_layout()
plt.show()

---
#### Calcula la tasa por 100,000 habitantes
##### Tasa por 100,000 habitantes
Mostrar el total de delitos en una entidad no nos sirve de mucho. Es mucho más útil calcular la tasa de incidencia delictiva por cada 100,000 habitantes

$$
tasa = \frac{delitos\space totales}{población} \times 100,000
$$

Esta tasa la podemos anualizar multiplicándola por un factor de 12
$$
tasa\space anualizada = tasa \times 12
$$

Población por entidad federativa según [la encuesta intercensal 2015](https://www.inegi.org.mx/programas/intercensal/2015/)

No tienes que descargar nada. Ya están los datos en la carpeta data

In [ ]:
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=";")
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})
pobs.head()

In [ ]:
df = df_delitos.merge(pobs, on='entidad', how='left')
df['tasa'] = (df['Delitos_totales'] / df['poblacion']) * 100000
df['tasa_anualizada'] = df['tasa'] * 12
df.head()
estados = ['Jalisco', 'CDMX', 'Chihuahua']
plt.figure(figsize=(12,6))
for estado in estados:
    df_estado = df[df['entidad'] == estado]
    fechas = pd.to_datetime(df_estado['Año'].astype(str) + '-' + df_estado['Mes'].astype(str))
    plt.plot(fechas, df_estado['tasa_anualizada'], label=estado)

plt.title('Tasa anualizada de delitos por 100,000 habitantes')
plt.xlabel('Fecha')
plt.ylabel('Tasa anualizada')
plt.legend()
plt.grid(True)
plt.show()